## Model Pre Training with augmentations and random labels


In [1]:
from datetime import datetime

import torch
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets

from models.model import ConvModel
from models.pre_trainer import PreTrainer

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1

In [3]:
color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)

composed_transforms = transforms.Compose([
    transforms.RandomApply([color_jitter], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor()])

In [4]:
train_dataset = datasets.CIFAR10('./datasets',  download=True, train=True, transform=composed_transforms)
g = torch.Generator()
g.manual_seed(0)

Files already downloaded and verified


In [5]:
train_loader_1 = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=False, pin_memory=True, drop_last=True, generator=g)

In [6]:
model = ConvModel(out_dim=10, dataset='CIFAR10')

C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader_1), eta_min=0,
                                                       last_epoch=-1)

with torch.cuda.device(gpu_index):
    pre_trainer = PreTrainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100,
                        log_dir=f'runs/Pretraining_CIFAR10_{datetime.now().strftime("%d-%m-%Y_%H-%M")}')
    pre_trainer.train(train_loader_1, shuffle=True)

  0%|          | 0/195 [00:00<?, ?it/s]C:\Users\kacpe\DataspellProjects\SimCLRPytorch\venv\lib\site-packages\torch\optim\lr_scheduler.py:807: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
 42%|████▏     | 81/195 [00:22<00:31,  3.67it/s]


KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), 'saved_models/pretraining/pretrained_cifar10.pth')